In [1]:
import numpy as np
import pandas as pd

import gensim
from gensim.models import Word2Vec

import re
import collections

import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_1=pd.read_csv(r"C:\Users\YUE\Desktop\interface1_total.csv",dtype={'Reviews':str})
data_2=pd.read_csv(r"C:\Users\YUE\Desktop\interface2_total.csv",dtype={'Reviews':str})
data_3=pd.read_csv(r"C:\Users\YUE\Desktop\interface3_total.csv",dtype={'Reviews':str})
data_4=pd.read_csv(r"C:\Users\YUE\Desktop\interface4_total.csv",dtype={'Reviews':str})
data_5=pd.read_csv(r"C:\Users\YUE\Desktop\interface5_total.csv",dtype={'Reviews':str})
data_6=pd.read_csv(r"C:\Users\YUE\Desktop\interface6_total.csv",dtype={'Reviews':str})


In [3]:
data=pd.concat([data_1,data_2,data_3,data_4,data_5,data_6])
print(len(data))

164031


In [4]:
new_data=data_6["Reviews"]
new_data.head()

0    video still open in browser den can tap to ope...
1    i can't load any of the video on my youtube ev...
2    after the update it get more worse come on you...
3    usually i love the youtube app but this curren...
4                   unable to load video .can't play .
Name: Reviews, dtype: object

In [5]:
#简单预处理
def clean_text(text):
    text= text.replace('\n'," ") #用空格代替新的行
    text=re.sub(r"-"," ",text)  #把用“-”连接的单词分开 （close-up分成close和 up）
    pure_text=''
    for letter in text:        #去除字母和空格
        letter=letter.lower()
        if letter.isalpha() or letter==' ':
            pure_text+=letter
    text=' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [6]:
docs=new_data.apply(lambda s:clean_text(s))
doclist=docs.values

In [7]:
sentences=[]
for sentence in doclist:
    raw_sentence=nltk.word_tokenize(sentence)
    sentences.append(raw_sentence)         

In [8]:
#word2vec模型建立
import multiprocessing
num_features=300                          #生成的单词向量的维数
min_word_count=1                          #最小字数的阈值
num_workers=multiprocessing.cpu_count()   #并行运行的线程数
context_size=3                            #上下文窗口长度
seed=1                                    #使得结果重现

word2vec_model=Word2Vec(sentences=sentences,
                        sg=1,
                        seed=seed,
                        workers=num_workers,
                        vector_size=num_features,
                        min_count=min_word_count,
                        window=context_size
                        )

In [9]:
testwords=["ui","interface","design","layout","gui","ux","clunky","redesign","aesthetic","navigation",
             "usability","sleek","appearance","aesthetically","intuitive","minimalistic",
              "ugly","slick","graphic","unintuitive"]

In [10]:
for i in range(len(testwords)):
    vector=word2vec_model.wv[testwords[i]]
    sim_words=word2vec_model.wv.most_similar(testwords[i])
    print(testwords[i])
    print(sim_words)

ui
[('interface', 0.751379132270813), ('layout', 0.7487566471099854), ('gui', 0.7257820963859558), ('clunky', 0.7021434307098389), ('ux', 0.7016051411628723), ('messy', 0.6965512633323669), ('unbelievably', 0.6917338967323303), ('format', 0.6867991089820862), ('redesign', 0.6819491386413574), ('setup', 0.6814700365066528)]
interface
[('layout', 0.7756385207176208), ('ui', 0.751379132270813), ('gui', 0.7513104677200317), ('design', 0.7164009809494019), ('setup', 0.7149686813354492), ('unbelievably', 0.7135179042816162), ('format', 0.7095189690589905), ('redesign', 0.7061988711357117), ('clunky', 0.7014716267585754), ('messy', 0.7012390494346619)]
design
[('layout', 0.7418424487113953), ('style', 0.7396385073661804), ('format', 0.7390375137329102), ('gui', 0.7281906008720398), ('setup', 0.7230738401412964), ('interface', 0.7164009213447571), ('ux', 0.7124044299125671), ('atrocious', 0.7068055868148804), ('horrid', 0.700555145740509), ('redesign', 0.6983566284179688)]
layout
[('format', 0